In [1]:
import os
import numpy as np

from nibabel.testing import data_path
import nibabel as nib

In [2]:
# Global variables
ROOT_DIR = "/net/i/broseau/Travail/3A/FouilleExtrcVisu/2019_ENSEIRB_Skull/"
DATA_DIR = '/tmp/broseau_data/nifti/'
TRAIN_DATA_PROPORTION = 0.8 # Proportion of data used for training

## Initializing dataset

In [3]:
# Getting all the data files
dataset_data, dataset_labels = [], []
i_data, i_label = 0, 0
for subdir, dirs, files in os.walk(DATA_DIR):
    for file in files:
        filename = os.path.join(subdir, file)
        if filename.endswith(".nii") and not "nifti_em" in filename:
            # Adding source data into dataset_data
            if not ("-labels" in filename):
                dataset_data.append(nib.load(filename).get_fdata())
                print("Added to train data:", i_data,":\t", filename)
                i_data += 1
            # Adding label data into dataset_labels
            else:
                dataset_labels.append(nib.load(filename).get_fdata())
                print("Added to label data:", i_label, ":\t", filename)
                i_label += 1
            
dataset_data, dataset_labels = np.array(dataset_data), np.array(dataset_labels)

# Verifying training data is valid
assert(dataset_data.shape == dataset_labels.shape)
print("Dataset shape: ", dataset_data.shape)

Added to train data: 0 :	 /tmp/broseau_data/nifti/527/Shank_527.ibw.nii
Added to label data: 0 :	 /tmp/broseau_data/nifti/527/Shank_527-labels.nii
Added to label data: 1 :	 /tmp/broseau_data/nifti/550/Shank_550c-labels.nii
Added to train data: 1 :	 /tmp/broseau_data/nifti/550/Shank_550c.ibw.nii
Added to train data: 2 :	 /tmp/broseau_data/nifti/497/Shank_497.ibw.nii
Added to label data: 2 :	 /tmp/broseau_data/nifti/497/Shank_497-labels.nii
Added to train data: 3 :	 /tmp/broseau_data/nifti/499/Shank_499.ibw.nii
Added to label data: 3 :	 /tmp/broseau_data/nifti/499/Shank_499-labels.nii
Added to train data: 4 :	 /tmp/broseau_data/nifti/496/Shank_496.ibw.nii
Added to label data: 4 :	 /tmp/broseau_data/nifti/496/Shank_496-labels.nii
Added to train data: 5 :	 /tmp/broseau_data/nifti/523/Shank_523.ibw.nii
Added to label data: 5 :	 /tmp/broseau_data/nifti/523/Shank_523-labels.nii
Added to label data: 6 :	 /tmp/broseau_data/nifti/522/Shank_522-labels.nii
Added to train data: 6 :	 /tmp/broseau_da

In [4]:
def splitTrainingData(data, proportion):
    split_index = int(len(data)*proportion)
    tmp = np.split(data, [split_index, len(data)])
    return tmp[0], tmp[1]

In [5]:
# Splitting data into testing and training sets
X_train, X_test  = splitTrainingData(dataset_data, TRAIN_DATA_PROPORTION)
Y_train, Y_test  = splitTrainingData(dataset_labels, TRAIN_DATA_PROPORTION)

# Verifying training data is valid
assert(X_train.shape == Y_train.shape and X_test.shape == Y_test.shape)
print("Training data:", X_train.shape[0], "\nTesting data: ", X_test.shape[0])

Training data: 14 
Testing data:  4


In [6]:
# Printing maximum value for each dataset
for i in Y_train:
    print(np.max(i))
for i in Y_test:
    print(np.max(i))

def removeUnusedLabels(dataset):
    f = np.vectorize(lambda x: 1 if x == 2. else 0, otypes=[np.int8])
    return f(dataset)

Y_train = removeUnusedLabels(Y_train)
Y_test  = removeUnusedLabels(Y_test)

# Verifying label data validity
assert np.max(Y_train) == 1., "Y_train has "+str(np.max(Y_train))+", expected 1."
assert np.max(Y_test) == 1., "Y_test has "+str(np.max(Y_test))+", expected 1."
assert(X_train.shape == Y_train.shape and X_test.shape == Y_test.shape)

2.0
2.0
2.0
2.0
2.0
4.0
2.0
2.0
2.0
0.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0


# Augmenting dataset

In [ ]:
#TODO

# Creating model
#498, 516, 523


In [17]:
#taken from https://github.com/zhixuhao/unet/blob/master/model.py
#TODO: verify shapes and network in itself

import skimage.io as io
import skimage.transform as trans
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as keras


def unet3D(pretrained_weights = None,input_size = (128,128,128,1)):
    inputs = Input(input_size)
    conv1 = Conv3D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv3D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling3D(pool_size=(2, 2, 2))(conv1)
    conv2 = Conv3D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv3D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling3D(pool_size=(2, 2, 2))(conv2)
    conv3 = Conv3D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv3D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling3D(pool_size=(2, 2, 2))(conv3)
    conv4 = Conv3D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv3D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling3D(pool_size=(2, 2, 2))(drop4)

    conv5 = Conv3D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv3D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv3D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling3D(size = (2,2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 4)
    conv6 = Conv3D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv3D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv3D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling3D(size = (2,2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 4)
    conv7 = Conv3D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv3D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv3D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling3D(size = (2,2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 4)
    conv8 = Conv3D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv3D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv3D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling3D(size = (2,2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 4)
    conv9 = Conv3D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv3D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv3D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv3D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs, conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    #model.summary()

    if(pretrained_weights):
        model.load_weights(pretrained_weights)

    return model

In [18]:
model = unet3D()